In [ ]:
import re
import string
import pandas as pd
from math import log
import os
import random
import hnswlib
import re
import string
from collections import Counter
from sklearn.cluster import KMeans
import pickle

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")

import pickle
SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

from transformers import AutoTokenizer, AutoModel
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch

import pandas as pd
import sys
print(sys.setrecursionlimit(5000))

In [ ]:
final_counts = []

In [ ]:
def clean_text(text):
    text = str(text).lower().strip() 
    text = re.sub("(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*?)", "", text)
    text = re.sub('[^a-z\s]+', "", text)
    text = text.translate (str.maketrans('', '', string.punctuation))
      # Remove multiple spaces in content
    text = re.sub('[!()-[]{};:\'"\,<>./?@#$%^&*_~–]+', '', text)
    # text = re.sub("\d+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

Overall, the function performs various text cleaning tasks, such as converting to lowercase, removing URLs, non-alphabetic characters, and punctuation marks, and reducing multiple spaces to a single space, to obtain a cleaned version of the input text suitable for further processing or analysis.

In [ ]:
all_roman_words = Counter()
pref = "/home/installer/ps/long_term/data/"

for file in os.listdir("/home/installer/ps/long_term/data/"):
    if not file.endswith("csv"):
        continue

    file_path = os.path.join("/home/installer/ps/long_term/data/", file)
    print(file, "here")
    
    with open(file_path, 'r') as f:
        df = pd.read_csv(f, usecols=[0, 21], header=None, low_memory=False)
        df.columns = ['reg_no', 'subject_content']
        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)

        df['subject_content_cleaned'] = df['subject_content'].apply(clean_text)
        for sent in df.subject_content_cleaned.values:
            res = re.findall(r'\w+', sent)
            all_roman_words.update(res)
        print(file, "Size of updated set", str(len(all_roman_words)))

    # print(file, "Size of updated set", str(len(all_roman_words)))
    


The code is reading multiple CSV files from a directory, extracting specific columns ('reg_no' and 'subject_content'), cleaning the text data in the 'subject_content' column using a function called 'clean_text', and then extracting all the individual words (excluding punctuation) from the cleaned text using regular expressions. These words are then added to a counter called 'all_roman_words', which keeps track of the frequency of each word across all the files. The code prints the name of each file being processed and the size of the updated 'all_roman_words' counter after processing each file.

In [ ]:

with open('all_roman_words.pkl', 'wb') as f:
    pickle.dump(all_roman_words, f, protocol=pickle.HIGHEST_PROTOCOL)
len(all_roman_words)


In [ ]:
def memo(f):
    "Memoize function f, whose args must all be hashable."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [ ]:
max_len = max(len(w) for w in all_roman_words.keys())


In [ ]:
def Pword(word, N=sum(all_roman_words.values())): 
    "Probability of `word`."
    return all_roman_words[word] / N

def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    return product(Pword(w) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

def splits(text, start=0, L=max_len):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]
    
@memo
def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

The segment(text) function utilizes dynamic programming to find the most probable segmentation of a given input text. It generates all possible splits, recursively calculates probabilities using Pwords(words) function, and returns the segmentation with the highest probability. It's a recursive implementation of Viterbi algorithm used in NLP tasks like POS tagging and named entity recognition.

In [ ]:
def increase_count(words, k):
    result = []
    for word in words:
        result += [word] * k
    return result


for k in list(all_roman_words):
    try:
        tmp = segment(k)
        if (len(tmp)>1):
            tmp = increase_count(tmp, all_roman_words[k])
            all_roman_words.update(tmp)
            del all_roman_words[k]
    except:
        print(k)

This code seems to be iterating through a dictionary called all_roman_words and attempting to segment each word into its constituent parts using a function called segment. If the resulting list has more than one element, it multiplies that list by a factor of all_roman_words[k] using a function called increase_count. It then updates the dictionary with the new segmented and multiplied list and deletes the original word from the dictionary. Overall, the code appears to be manipulating a dictionary of Roman words, breaking them down into their components, and multiplying the components based on the frequency of the original word.

In [ ]:

    # del all_roman_words[k]
    

In [ ]:
all_roman_words.most_common(10)

In [ ]:
with open('all_roman_words_segmented.pkl', 'wb') as f:
    pickle.dump(all_roman_words, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/home/installer/ps/hnsw_rewording/all_roman_words_segmented.pkl', 'rb') as f:
    all_roman_words = pickle.load(f)

In [ ]:
all_roman_words.most_common(10)

In [ ]:
def fit_hnsw_index(features, ef=100, M=32):
    num_elements = len(features)
    labels_index = np.arange(num_elements)
    EMBEDDING_SIZE = len(features[0])
    
    p = hnswlib.Index(space='l2', dim=EMBEDDING_SIZE)# possible space options are l2, cosine or ip
    p.init_index(max_elements=num_elements, ef_construction=ef, M=M)
    int_labels = p.add_items(features, labels_index)
    p.set_ef(-1) 
    return p

def freq(sent):
    arr = np.zeros(26)
    for letter in sent:
        if 97 <= ord(letter) <= 122:
            # print("here")
            arr[ord(letter) - 97] += 1 
    #arr[::-1].sort()
    
    return arr / np.linalg.norm(arr) #normalization

The first function, "fit_hnsw_index," trains an HNSW index using a set of input features. The index is constructed using the "l2" space and is initialized with specified values of "ef" and "M." The function returns the trained index. The second function, "freq," calculates the frequency of each alphabet in a given input sentence and returns a normalized array of these frequencies.


In [ ]:
df = pd.DataFrame(all_roman_words.keys(), columns= ['word'])
df['histograms'] = df['word'].apply(freq)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
with open('df_histograms.pkl', 'wb') as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/home/installer/ps/hnsw_rewording/df_histograms.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
K = 50 #nearest neighbours you wish to find for each grievance
l2_param = 0.001 #threshold for l2 distance
min_docs = 1 #minimum number of grievances in a set to call it a bulk

In [ ]:
X = [item.tolist() for item in df['histograms'].values]

p = fit_hnsw_index(X, ef=K*10)

def find_nearest(index, word, k=50):
    ann_neighbor_indices, ann_distances = index.knn_query(freq(word), k)
    out = np.array([df.loc[i, "word"].values for i in ann_neighbor_indices])
    ann_distances = np.array(ann_distances)
    return out

primary_dict = {w: find_nearest(p, w) for w in all_roman_words.keys()}


This code is creating a primary dictionary of nearest neighbors for each word in a set of input data. The input data consists of histograms stored in a Pandas DataFrame. First, the histograms are converted to a list of lists format. Then, an HNSW index is fit to the list of histograms. Finally, a function is defined that finds the nearest neighbors for a given word based on the HNSW index. The nearest neighbors are returned as an array of words, with the distance to each neighbor also provided. The primary dictionary is created by calling this function for each word in a pre-defined set of words, creating a dictionary where each word maps to its nearest neighbors.

In [ ]:
with open('primary_dict.pkl', 'wb') as f:
    pickle.dump(primary_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/home/installer/ps/hnsw_rewording/primary_dict.pkl', 'rb') as f:
    primary_dict = pickle.load(f)

In [ ]:
def known(words, lwords): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in lwords)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


def candidates(word, lwords): 
    "Generate possible spelling corrections for word."
    return( known(edits1(word), lwords) or known(edits2(word), lwords) or known(word, lwords) or [word])


def P(word, N=sum(all_roman_words.values())): 
    "Probability of `word`."
    return all_roman_words[word] / N

def correction(word, lwords): 
    "Most probable spelling correction for word."
    return max(candidates(word, lwords), key=P)

This code defines functions for spelling correction using the edit distance algorithm. It generates a list of possible spelling corrections for a given word, ranks them by their probability of appearing in a dictionary, and returns the correction with the highest probability. The code uses a dictionary of known words to calculate probabilities and edit distance functions to generate possible corrections.

In [ ]:
with open('tmp.pkl', 'wb') as f:
    pickle.dump(tmp, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('/home/installer/ps/hnsw_rewording/tmp.pkl', 'rb') as f:
    tmp = pickle.load(f)
tmp

In [ ]:
tmp = {key: candidates(key, primary_dict[key]) for key in primary_dict.keys()}


english_words = set()
with open("word.list") as f:
    for line in f.readlines():
        line = line.strip()
        english_words.add(line)
        
letters    = 'abcdefghijklmnopqrstuvwxyz'
for l in letters:
    if l=='a':
        continue
    try:
        # del WORDS[l]
        del tmp[l]
    except:
        print(l, "doesn't exist")
        
def build_word_mapping(dictionary):
    word_mapping = {}
    for key, values in dictionary.items():
        for value in values:
            words = value.split()
            for word in words:
                if word not in word_mapping:
                    word_mapping[word] = key
    return word_mapping

word_mapping = build_word_mapping(tmp)

def correct_word(word, word_mapping):
    corrected_word = word
    if word in word_mapping:
        corrected_word = word_mapping[word]
    return corrected_word

The code reads a list of English words and creates a dictionary mapping letters to sets of words containing that letter. It then removes all sets of words containing the letter 'a'. Next, it creates a word mapping dictionary by assigning each word to its corresponding set of letters, and finally defines a function to correct words based on this mapping.

In [ ]:
with open('word_mapping.pkl', 'wb') as f:
    pickle.dump(word_mapping, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
correct_word("kya", word_mapping)

In [ ]:
word_mapping

In [ ]:

# Open the file in binary mode
with open('trans_hindi.pickle', 'rb') as f:
    # Load the object from the file
    trans_hindi = pickle.load(f)

In [ ]:
trans_hindi = set(trans_hindi.values())
len(trans_hindi)

final_wording = dict()

# Function to get the correct word using word_mapping
def get_correct_word(word, mapping):
    while word != mapping.get(word, word):
        word = mapping[word]
    return word

for v in word_mapping.keys():
    if v not in english_words and v not in trans_hindi:
        w = get_correct_word(v, word_mapping)
        if w in final_wording:
            final_wording[v] = final_wording[w]
        else:
            final_wording[v] = w



The code first creates a set of Hindi translations, and then creates an empty dictionary to store the final word mapping. It defines a function to recursively look up the correct word using the word mapping. It then iterates over the keys in the word mapping, and for each key that is not an English word or a Hindi translation, it uses the get_correct_word function to find the correct word, and stores it in the final_wording dictionary. If the correct word is already in the dictionary, it maps the current word to the same value.

In [ ]:
final_wording


In [ ]:
with open('final_wording.pkl', 'wb') as f:
    pickle.dump(final_wording, f, protocol=pickle.HIGHEST_PROTOCOL)